## Geração de Notícias utilizadndo GPT-2

Neste notebook iremos realizar treinamento (finetunning) do modelo GPT-2 treinado em PT-BR para gerar títulos de notícias de economia.

In [ ]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 49.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.8 MB/s eta 0:00:00


In [ ]:
!nvidia-smi

Tue Oct 17 00:00:45 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Preparação do dataset utilizando ``TextDataset``

A suite do huggingface contém funções específicas para utilização de dataset textual.

O conteúdo é carregado do Google Drive, e divido em conjunto de treinamento e teste sendo que 85% do conjunto é destinado a treinamento e 15% para teste

In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
text_file = open("/content/drive/MyDrive/gpt2-noticias/dataset_full_preprocessed_labeled.txt", "r")
#text_file = open("/content/drive/MyDrive/gpt2-noticias/dataset_pos.txt", "r")
#text_file = open("/content/drive/MyDrive/gpt2-noticias/dataset_neg.txt", "r")

lines = text_file.readlines()
print(lines[0:10])
print(len(lines))
text_file.close()

['Na disputa loira, Devassa levou a melhor\n', 'Espumante quer vender como cerveja\n', 'Cinco novas marcas de cerveja na praça\n', 'Mea culpa, contradições & mais Devassa\n', 'Nizan Guanaes deixa o comando da Africa\n', 'Cade vive um clima de guerra civil\n', 'AmBev amplia fábrica no Amazonas\n', 'Heineken se insinua na guerra das cervejas\n', 'Brasil ganha cerveja de mosteiro\n', 'Correção: governo estuda correção da tabela do IRPF\n']
141388


In [ ]:
import re
import json
from sklearn.model_selection import train_test_split

#with open('recipes.json') as f:
#    data = json.load(f)

def build_text_files(data_json, dest_path):
    f = open(dest_path, 'w')
    data = ''
    for texts in data_json:
        summary = str(texts).strip()
        summary = re.sub(r"\s", " ", summary)
        data += summary + "  "
    f.write(data)

train, test = train_test_split(lines,test_size=0.15)

train_path = 'train_dataset.txt'
test_path = 'test_dataset.txt'

build_text_files(train,train_path)
build_text_files(test,test_path)

print("Train dataset length: "+str(len(train)))
print("Test dataset length: "+ str(len(test)))

Train dataset length: 120179
Test dataset length: 21209


### Carregamento de tokenizador
Para o projeto, utilizaremos o tokenizador do modelo `pierreguillou/gpt2-small-portuguese` disponível no [huggingface](https://huggingface.co/pierreguillou/gpt2-small-portuguese).

In [ ]:
from transformers import TextDataset,DataCollatorForLanguageModeling
from transformers import AutoTokenizer

def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)

    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator



In [ ]:
start_from_epoch = False
project_name = 'gpt2-noticias'

model_name = "pierreguillou/gpt2-small-portuguese"

model_path = '/content/drive/MyDrive/gpt2-noticias/pierreguillou/gpt2-small-portuguese/2023_10_06_1511/epoch_'+str(start_from_epoch)
model_path = '/content/drive/MyDrive/gpt2-noticias/pierreguillou/gpt2-small-portuguese/'
#model_path = 'H:\My Drive/gpt2-noticias/pierreguillou/gpt2-small-portuguese/2023_10_06_1511/epoch_'+str(start_from_epoch)

(model_name, model_path)

('pierreguillou/gpt2-small-portuguese',
 '/content/drive/MyDrive/gpt2-noticias/pierreguillou/gpt2-small-portuguese/')

In [ ]:
if(os.path.isdir('/content/drive/MyDrive/gpt2-noticias/') == False ):
  os.makedirs('/content/drive/MyDrive/gpt2-noticias/')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Criação de pastas para resultados.

In [ ]:
from datetime import date
from datetime import datetime

date_str = datetime.now().strftime("%Y_%m_%d_%H%M")

model_drive_dir = model_path

if(os.path.isdir(model_drive_dir) == False):
  model_drive_dir = os.path.join('/content/drive/MyDrive/gpt2-noticias/', model_name)

if(start_from_epoch == False):
    model_drive_dir = os.path.join(model_drive_dir, date_str)
model_drive_dir

'/content/drive/MyDrive/gpt2-noticias/pierreguillou/gpt2-small-portuguese/2023_10_17_0001'

A seguir, carregamos o modelo, definimos os métodos para treinamento, geração e gravação dos títulos gerados.

In [ ]:
from transformers import Trainer, TrainingArguments,AutoModelWithLMHead

def load_model(model_name,model_drive_dir, data_collator, train_dataset, test_dataset):
  model = AutoModelWithLMHead.from_pretrained(model_name)

  training_args = TrainingArguments(
      output_dir=model_drive_dir, #The output directory
      overwrite_output_dir=True, #overwrite the content of the output directory
      num_train_epochs=10, # number of training epochs
      per_device_train_batch_size=32, # batch size for training
      per_device_eval_batch_size=64,  # batch size for evaluation
      eval_steps = 200, # Number of update steps between two evaluations.
      save_steps=400, # after # steps model is saved
      warmup_steps=200,# number of warmup steps for learning rate scheduler
      prediction_loss_only=True,
  )

  return Trainer(
      model=model,
      args=training_args,
      data_collator=data_collator,
      train_dataset=train_dataset,
      eval_dataset=test_dataset,
  )

In [ ]:
from transformers import pipeline

def generate_noticias(model_drive_dir, model_name, qtde_noticias = 10):
  print("Gerando "+str(qtde_noticias)+" noticias com o modelo: "+model_drive_dir+', tokenizer='+model_name)
  base_str =  '  '
  noticias = []

  gerador_noticias = pipeline('text-generation', model=model_drive_dir, tokenizer=model_name)

  while len(noticias) < qtde_noticias:
    noticias_geradas = gerador_noticias(base_str)[0]['generated_text'].split('  ')
    if(len(noticias_geradas) == 0):
      noticias_geradas = gerador_noticias(base_str)[0]['generated_text'].split('.')

    for n in noticias_geradas:
      if(len(n.strip()) < 3):
        continue;
      noticias.append(n.strip());
      print(n.strip());

    base_str = noticias.pop();
    if(len(noticias)<1):
      break


  return noticias

In [ ]:
def save_noticias_geradas(model_drive_dir, noticias):
  file = open(os.path.join(model_drive_dir, 'noticias_geradas_'+date_str+'.txt'),'w+')

  for n in noticias:
    #print(n)
    file.write(n)
    file.write("\n")

  file.close()

def save_metrics(model_drive_dir, metricas):
  file = open(os.path.join(model_drive_dir, 'metricas_'+date_str+'.txt'),'w+')

  file.write(str(metricas))

  file.close()

## Treinamento do Modelo

O modelo pode ser treinado do zero ou continuar o treinamento anterior.

In [ ]:
start_from_epoch=False
if(start_from_epoch != False):
    saved_model_drive_dir = os.path.join(model_drive_dir, 'epoch_'+str(start_from_epoch))
else:
    saved_model_drive_dir = os.path.join(model_drive_dir, 'epoch_0')

#if(os.path.isdir(model_drive_dir) == False):
#    saved_model_drive_dir = model_name

(model_drive_dir, saved_model_drive_dir)


('/content/drive/MyDrive/gpt2-noticias/pierreguillou/gpt2-small-portuguese/2023_10_17_0001',
 '/content/drive/MyDrive/gpt2-noticias/pierreguillou/gpt2-small-portuguese/2023_10_17_0001/epoch_0')

In [ ]:
trainer = load_model(model_name,saved_model_drive_dir, data_collator, train_dataset, test_dataset)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1499: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [ ]:
for i in range(start_from_epoch+1,start_from_epoch+2):
  iteration_model_path = os.path.join(model_drive_dir, "epoch_" + str(i))

  trainer.train()

  print("Saving model to: "+iteration_model_path)
  trainer.save_model (iteration_model_path)

  metrics = str(trainer.evaluate())
  print(metrics)
  save_metrics(iteration_model_path, metrics)

  noticias = generate_noticias(iteration_model_path, model_name, 500)
  save_noticias_geradas(iteration_model_path, noticias)


start_from_epoch=+5

Step,Training Loss
500,4.262400
1000,3.664100
1500,3.500900
2000,3.389000
2500,3.320400
3000,3.262600
3500,3.223100
4000,3.195200


Saving model to: /content/drive/MyDrive/gpt2-noticias/pierreguillou/gpt2-small-portuguese/2023_10_17_0001/epoch_1


{'eval_loss': 3.3646411895751953, 'eval_runtime': 26.7392, 'eval_samples_per_second': 90.953, 'eval_steps_per_second': 1.421, 'epoch': 10.0}
Gerando 500 noticias com o modelo: /content/drive/MyDrive/gpt2-noticias/pierreguillou/gpt2-small-portuguese/2023_10_17_0001/epoch_1, tokenizer=pierreguillou/gpt2-small-portuguese


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Bovespa opera em queda, abaixo dos 62 mil pontos
Presidente do Santander nega acusações de improbidade de cliente
Ibovespa fecha pregão com alta de 3,31%, aos 55.642 pontos
Ibovespa segue mercados


...

Ibovespa recua 1,32%, aos 67 desabrigados em PE após temporal
Bandidos explodem agência do Banco do Brasil em Tucuruí
Receita libera consulta a 5.928 lotes residuais do PIS pelo 11º dia


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Receita libera consulta a 5.928 lotes residuais do PIS pelo 11º dia
Ibovespa recupera os 68 mil pontos, mas recua após 3 dias de queda
Em busca de novas propostas, Bolsonaro defende inclusão digital e a defesa do


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Em busca de novas propostas, Bolsonaro defende inclusão digital e a defesa do consumidor
Bolsonaro diz esperar eleições no Brasil em 2017
Justiça manda banco liberar verba para tentar limpar nome no cadastro de consumidores
Grupo PPI quer levar de volta R$1 bilhão
Grupo PPI quer levar de volta R$1 bilhão para o FGTS
Bovespa fecha em baixa, acompanhando bom humor externo
Polícia investiga suposta fraude em concurso público de medicina do RJ
Brasil tem saída de US$ 2,2


In [ ]:
  noticias = generate_noticias(iteration_model_path, model_name, 500)
  save_noticias_geradas(iteration_model_path, noticias)

Gerando 500 noticias com o modelo: /content/drive/MyDrive/gpt2-noticias/pierreguillou/gpt2-small-portuguese/2023_10_17_0001/epoch_1, tokenizer=pierreguillou/gpt2-small-portuguese


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Dólar opera instável nesta sexta, após Fed sinalizar abertura da bolsa
Receita abre consulta ao 2º lote de restituições do Imposto de Renda 2017 nesta segunda
Moradores de Itu têm que pagar para ver o Carnaval; confira como


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Moradores de Itu têm que pagar para ver o Carnaval; confira como participar
No Rio, banco de sangue do Hospital-SP recebe doação de doações
Bovespa cede 0,40% e encerra a semana em queda
Homem é preso


...

Quadrilha explode caixas eletrônicos de bancos e atira contra policiais; leia declaração de ataque a São Luís
Com atraso nas contas do presidente, caixa tem alta de 5,1%
Bovespa fecha em alta pelo terceiro dia seguido com mercado à


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Bovespa fecha em alta pelo terceiro dia seguido com mercado à espera de relatório do BC dos EUA
Bovespa encerra semana em leve queda puxado por Vale e Petrobras
Nomes do grupo B e outros oito presos, de acordo com Ministério Público


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Nomes do grupo B e outros oito presos, de acordo com Ministério Público italiano
Bovespa ganha 0,67% no fechamento, impulsionado por Wall Street
Agência do Banco do Japão é alvo de criminosos em Macapá
Banco de leite materno
Banco de leite materno precisa de doações e doações de órgãos
Petrobras dispara com possível venda de fatia na Vale
BNDES e bancos criam canal de investimento e mantêm financiamentos nos Estados
Governo anuncia redução de juros em 15% para bancos e comércio no futuro
